In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from itertools import chain
from matplotlib import pyplot as plt
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
#import keras_tuner as kt
df = pd.read_csv("drive/MyDrive/Engineer's Project/output_pln_usd.csv")

In [ ]:
df.pop('usa_cpi')
df.pop('pol_cpi')
df.pop('usa_inter')
df.pop('pol_inter')
df.pop('Date')

0       2010.11.15
1       2010.11.16
2       2010.11.17
3       2010.11.18
4       2010.11.19
           ...    
3537    2022.03.27
3538    2022.03.28
3539    2022.03.29
3540    2022.03.30
3541    2022.03.31
Name: Date, Length: 3542, dtype: object

In [ ]:
scaler = MinMaxScaler()
df_numpy = df.to_numpy() 
scaler = scaler.fit(df_numpy[:3000])
df_scalled = scaler.transform(df_numpy)
df_scalled = pd.DataFrame(df_scalled, columns=[
  'Opening', 'High', 'Low', 'Closing','Momentum', 'Range', 'ohlc'])

In [ ]:
df_train = df_scalled[0:2500]
df_val = df_scalled[2500:3000] #300
df_game = df_scalled[3001:]

In [ ]:
lookback = 30 #15
step = 1
delay = 30 #0
batch_size = 128

In [ ]:
float_data = np.array(df_scalled).astype('float32')
float_train_data = np.array(df_train).astype('float32')
float_val_data = np.array(df_val).astype('float32')
float_game_data = np.array(df_game).astype('float32')

In [ ]:
def generator(data, lookback, delay, min_index, max_index,shuffle=False, batch_size=128, step=1):
  if max_index is None:
    max_index = len(data) - delay - 1
  i = min_index + lookback
  while 1:
    if shuffle:
      rows = np.random.randint(
        min_index + lookback, max_index, size=batch_size)
    else:
      if i + batch_size >= max_index:
        i = min_index + lookback
      rows = np.arange(i, min(i + batch_size, max_index))
      i += len(rows)
    samples = np.zeros((len(rows),lookback // step,data.shape[-1]))
    targets = np.zeros((len(rows),))
    for j, row in enumerate(rows):
      indices = range(rows[j] - lookback, rows[j], step)
      samples[j] = data[indices]
      targets[j] = data[rows[j] + delay][3] 
    yield samples, targets

In [ ]:
train_gen = generator(float_train_data,
lookback=lookback,
delay=delay,
min_index=0,
max_index=2499-delay,
#shuffle=True,
step=step,
batch_size=batch_size)

In [ ]:
val_gen = generator(float_val_data,
lookback=lookback,
delay=delay,
min_index=0,
max_index=500-delay,
step=step,
batch_size=batch_size)

In [ ]:
test_gen = generator(float_game_data,
lookback=lookback,
delay=delay,
min_index=0,
max_index=540-delay,
step=step,
batch_size=batch_size)

In [ ]:
train_steps = (2500 - lookback)
val_steps = (500 - lookback)
test_steps = (541 - lookback)

In [ ]:
model = tf.keras.models.load_model("drive/MyDrive/Engineer's Project/Final Models/USDPLN/longer_hybrid_usd_pln.h5")

In [ ]:
eval = model.evaluate(test_gen, steps = test_steps)

511/511 [==============================] - 19s 36ms/step - loss: 0.0526


In [ ]:
eval

0.052598342299461365

In [ ]:
correct_picks = 0
value = 0
for i in range(0, 541):
  last15 = float_train_data[-lookback:]
  last15 = last15.reshape(1,15,7)
  prediction = model.predict(last15)
  last_val = last15[0][lookback-1][3]
  actual_val = float_game_data[0][3]
  if((prediction > last_val) and (actual_val > last_val)):
    correct_picks+=1
    value +=  100 *(actual_val - last_val)
  elif((prediction < last_val) and (actual_val < last_val)):
    correct_picks+=1
    value += 100 * (last_val - actual_val)
  else:
    value -= 100 * abs(actual_val - last_val)
  float_train_data = np.vstack((float_train_data, float_game_data[0]))
  float_game_data = np.delete(float_game_data, 0, 0)

1/1 [==============================] - 0s 21ms/step


LONGER:

In [ ]:
correct_picks = 0
value = 0
for i in range(len(float_game_data-lookback)): #len(float_train_data - lookback - delay
  if len(float_game_data) <= 30:
    break
  last = float_train_data[-lookback:]
  last = last.reshape(1,30,7)
  prediction = model.predict(last)
  last_val = last[0][lookback-1][3]
  actual_val = float_game_data[29][3]
  if((prediction > last_val) and (actual_val > last_val)):
    correct_picks+=1
    value += 100 * (actual_val - last_val)
  elif((prediction < last_val) and (actual_val < last_val)):
    correct_picks+=1
    value += 100 * (last_val - actual_val)
  else:
    value -= 100 * abs(actual_val - last_val)
  float_train_data = np.vstack((float_train_data, float_game_data[0]))
  float_game_data = np.delete(float_game_data, 0, 0)

1/1 [==============================] - 0s 22ms/step


In [ ]:
len(float_game_data)-lookback-delay

481

In [ ]:
correct_picks

297

In [ ]:
value

446.73221707344055

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 120)               61440     
                                                                 
 dense (Dense)               (None, 160)               19360     
                                                                 
 dense_1 (Dense)             (None, 1)                 161       
                                                                 
Total params: 80,961
Trainable params: 80,961
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#GRU
def return_model():
  model = tf.keras.models.Sequential()
  model.add(layers.GRU(140,
    activation='sigmoid',
    #dropout=0.1,
    #recurrent_dropout=0.1,
    return_sequences=True,
    input_shape=(None, float_train_data.shape[-1])))
  model.add(layers.Dense(120,
    activation='sigmoid',))
  model.add(layers.GRU(2,
    activation='sigmoid',
    input_shape=(None, float_train_data.shape[-1])))
  model.add(layers.Dense(1,
    activation='sigmoid'))
  model.compile(optimizer= tf.keras.optimizers.RMSprop(0.01), loss='mae')
  return model

In [ ]:
#LSTM
def return_model():
  model = tf.keras.models.Sequential()
  model.add(layers.LSTM(120,
    activation='sigmoid',
    #dropout=0.1,
    #recurrent_dropout=0.1,
    #return_sequences=True,
    input_shape=(None, float_train_data.shape[-1])))
  model.add(layers.Dense(160,
    activation='sigmoid',))
  model.add(layers.Dense(1,
    activation='sigmoid'))
  model.compile(optimizer= tf.keras.optimizers.RMSprop(0.01), loss='mae')
  return model

In [ ]:
#HYBRID
def return_model():
  model = tf.keras.models.Sequential()
  model.add(layers.LSTM(120,
    activation='sigmoid',
    #dropout=0.1,
    #recurrent_dropout=0.1,
    return_sequences=True,
    input_shape=(None, float_train_data.shape[-1])))
  model.add(layers.GRU(5,
    activation='sigmoid',
    #dropout=0.1,
    #recurrent_dropout=0.1,
    input_shape=(None, float_train_data.shape[-1])))
  model.add(layers.Dense(160,
    activation='sigmoid',))
  model.add(layers.Dense(1,
    activation='sigmoid'))
  model.compile(optimizer= tf.keras.optimizers.RMSprop(0.01), loss='mae')
  return model

In [ ]:
correct_peaks = 0
value = 0
for i in range(0, 541):##441
  last15 = float_train_data[-lookback:]
  last15 = last15.reshape(1,15,7)
  prediction = model.predict(last15)
  last_val = last15[0][lookback-1][3]
  actual_val = float_game_data[0][3]
  if((prediction > last_val) and (actual_val > last_val)):
    correct_peaks+=1
    value += 100 * (actual_val - last_val)
  elif((prediction < last_val) and (actual_val < last_val)):
    correct_peaks+=1
    value += 100 * (last_val - actual_val)
  else:
    value -= 100 * abs(actual_val - last_val)
  float_train_data = np.vstack((float_train_data, float_game_data[0]))
  float_game_data = np.delete(float_game_data, 0, 0)

  if i == 150 or i == 300 or i == 450:
    print(i)
    del model
    del train_gen
    model = return_model()
    train_gen = generator(float_train_data,
      lookback=lookback,
      delay=delay,
      min_index=0,
      max_index=len(float_train_data) - 1,
      #shuffle=True,
      step=step,
      batch_size=batch_size)
    model.fit(train_gen,
      steps_per_epoch=100,
      epochs=200,
      validation_data=val_gen,
      validation_steps=val_steps)

1/1 [==============================] - 0s 19ms/step
150
Epoch 1/200
100/100 [==============================] - 17s 142ms/step - loss: 0.1805 - val_loss: 0.0768
Epoch 2/200
100/100 [==============================] - 13s 132ms/step - loss: 0.1747 - val_loss: 0.0664
Epoch 3/200
100/100 [==============================] - 13s 132ms/step - loss: 0.1716 - val_loss: 0.0633
Epoch 4/200
100/100 [==============================] - 13s 132ms/step - loss: 0.1449 - val_loss: 0.0419
Epoch 5/200
100/100 [==============================] - 14s 140ms/step - loss: 0.1333 - val_loss: 0.0725
Epoch 6/200
100/100 [==============================] - 13s 133ms/step - loss: 0.1286 - val_loss: 0.0953
Epoch 7/200
100/100 [==============================] - 13s 132ms/step - loss: 0.1441 - val_loss: 0.0926
Epoch 8/200
100/100 [==============================] - 13s 132ms/step - loss: 0.1518 - val_loss: 0.0832
Epoch 9/200
100/100 [==============================] - 13s 133ms/step - loss: 0.1622 - val_loss: 0.0554
Epoch 10

In [ ]:
correct_peaks

272

In [ ]:
value

-6.431758403778076

In [ ]:
eval = model.evaluate(test_gen, steps = test_steps)

526/526 [==============================] - 8s 16ms/step - loss: 0.0088


In [ ]:
eval

0.008805087767541409

LONGER GAME:

In [ ]:
correct_picks = 0
value = 0
for i in range(0,len(float_game_data-lookback)):
  if len(float_game_data) <= 30:
    break
  last = float_train_data[-lookback:]
  last = last.reshape(1,30,7)
  prediction = model.predict(last)
  last_val = last[0][lookback-1][3]
  actual_val = float_game_data[29][3]
  if((prediction > last_val) and (actual_val > last_val)):
    correct_picks+=1
    value += 100 * (actual_val - last_val)
  elif((prediction < last_val) and (actual_val < last_val)):
    correct_picks+=1
    value += 100 * (last_val - actual_val)
  else:
    value -= 100 * abs(actual_val - last_val)
  float_train_data = np.vstack((float_train_data, float_game_data[0]))
  float_game_data = np.delete(float_game_data, 0, 0)

  if i == 150 or i == 300 or i == 450:
    print(i)
    del model
    del train_gen
    model = return_model()
    train_gen = generator(float_train_data,
      lookback=lookback,
      delay=delay,
      min_index=0,
      max_index=len(float_train_data) - delay - 1,
      #shuffle=True,
      step=step,
      batch_size=batch_size)
    model.fit(train_gen,
      steps_per_epoch=100,
      epochs=200,
      validation_data=val_gen,
      validation_steps=val_steps)

1/1 [==============================] - 0s 26ms/step
150
Epoch 1/200
100/100 [==============================] - 27s 246ms/step - loss: 0.1761 - val_loss: 0.0472
Epoch 2/200
100/100 [==============================] - 28s 285ms/step - loss: 0.1664 - val_loss: 0.0537
Epoch 3/200
100/100 [==============================] - 25s 249ms/step - loss: 0.1607 - val_loss: 0.0603
Epoch 4/200
100/100 [==============================] - 24s 243ms/step - loss: 0.1633 - val_loss: 0.0501
Epoch 5/200
100/100 [==============================] - 24s 241ms/step - loss: 0.1612 - val_loss: 0.0503
Epoch 6/200
100/100 [==============================] - 24s 241ms/step - loss: 0.1627 - val_loss: 0.0474
Epoch 7/200
100/100 [==============================] - 24s 242ms/step - loss: 0.1666 - val_loss: 0.0478
Epoch 8/200
100/100 [==============================] - 24s 243ms/step - loss: 0.1662 - val_loss: 0.0489
Epoch 9/200
100/100 [==============================] - 24s 241ms/step - loss: 0.1638 - val_loss: 0.0675
Epoch 10

In [ ]:
correct_picks

278

In [ ]:
value

169.76801753044128

In [ ]:
eval = model.evaluate(test_gen, steps = test_steps)

511/511 [==============================] - 18s 35ms/step - loss: 0.0454


In [ ]:
eval

0.0453614667057991